# Logging with AzureML SDK and MLFlow

In [2]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient


credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "<Subscription ID>"
    resource_group = "<ResourceGroup Name>"
    workspace = "<<WorkspaceName>"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)
print(ml_client)

We could not find config.json in: . or in its parent directories. 
MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7fb552d55220>,
         subscription_id=25758354-2fe7-426a-be0e-1ad20058340d,
         resource_group_name=azureml-live,
         workspace_name=ftaliveazureml)


In [3]:
import mlflow

# Obtain the tracking URL from MLClient and set the MLFlow tracking URI
MLFLOW_TRACKING_URI = ml_client.workspaces.get(
    name=ml_client.workspace_name
).mlflow_tracking_uri

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

print("\nCurrent tracking uri: {}".format(mlflow.get_tracking_uri()))



Current tracking uri: azureml://southeastasia.api.azureml.ms/mlflow/v1.0/subscriptions/25758354-2fe7-426a-be0e-1ad20058340d/resourceGroups/azureml-live/providers/Microsoft.MachineLearningServices/workspaces/ftaliveazureml


In [11]:
from mlflow.tracking.client import MlflowClient

# Initialize MLFlow client
mlflow_client = MlflowClient()

# Create a new experiment if one doesn't already exist
mlflow.set_experiment("mlflow-experiment")

# Start the run, log metrics, end the run
mlflow_run = mlflow.start_run()
mlflow.log_metric('mymetric', 1)
mlflow.end_run()

### Integrating with a Training script to log Metrics and Parameters

In [12]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoLars
from sklearn import datasets

with mlflow.start_run():
    mlflow.sklearn.autolog()
    # Normal sklearn training code
    alpha_param = 0.9
    diabetes = datasets.load_diabetes()
    X = diabetes.data
    y = diabetes.target
    pipe = make_pipeline(StandardScaler(), LassoLars(alpha=alpha_param))
    pipe.fit(X, y)
    # Track using the MLflow API
    mlflow.log_metric("custom_metric", 0.9)
    # MLflow already tracks lassolars__alpha
    mlflow.log_param("alpha", alpha_param)

2022/06/15 10:05:15 WARNING mlflow.utils: Truncated the value of the key `steps`. Truncated value: `[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('lassolars', LassoLars(alpha=0.9, copy_X=True, eps=2.220446049250313e-16, fit_intercept=True,
          fit_path=True, max_iter=500, normalize=True, positive=False,
  ...`
2022/06/15 10:05:21 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: ('cannot represent an object', OrderedDict([('name', 'mlflow-env'), ('channels', ['conda-forge']), ('dependencies', ['python=3.8.5', 'pip', {'pip': ['mlflow', 'cloudpickle==1.6.0', 'lz4==4.0.0', 'psutil==5.9.0', 'scikit-learn==0.22.1', 'uuid==1.30']}])]))
